In [1]:
import json
import pandas as pd
import numpy as np
import a1chemy as a1
import a1chemy.indicators as indicators
import a1chemy.data_source as data_source
from tqdm.notebook import tqdm
import random
import time

# sectors data
sectors_list = a1.read_json_from_file('/Users/line/workspace/freedom/A1chemy/data/sw_sector.json')
stock_sector_map = {}
sectors = set()
for stock in sectors_list:
    sector = stock['sector']
    stock_sector_map[stock['symbol']] = sector
    sectors.add(sector)

# stocks
stocks = a1.read_json_from_file('/Users/line/workspace/freedom/A1chemy/data/csi_800.json')
sci_300_sector_map = {}
for stock in stocks:
    sector = stock_sector_map[stock['symbol']]
    sector_list = sci_300_sector_map.setdefault(sector, [])
    sector_list.append(stock)

sci_300_sector_count_map = {}
for k, v in sci_300_sector_map.items():
    sci_300_sector_count_map[k] = len(v)

for k, v in sci_300_sector_count_map.items():
    company_name_list = ",".join([d['name'] for d in sci_300_sector_map[k]])
    print(k + "," + str(v) + ",   " + company_name_list)

# scrap data
xueqiu_client = data_source.XueQiuDataParser()
stock_candle_data_map = {}
for stock in tqdm(stocks):
    time.sleep(random.uniform(0.1,0.3))
    symbol = stock['symbol']
    stock_ticks = xueqiu_client.history(symbol=symbol, period = 'day', count=100)
    stock_ticks.raw_data['MA20'] = indicators.sma(data=stock_ticks.close(), day=20)
    stock_candle_data_map[symbol] = stock_ticks

银行,34,   浦发银行,华夏银行,民生银行,招商银行,无锡银行,江苏银行,杭州银行,西安银行,南京银行,渝农商行,常熟银行,兴业银行,北京银行,上海银行,农业银行,交通银行,工商银行,长沙银行,邮储银行,光大银行,成都银行,紫金银行,浙商银行,建设银行,中国银行,贵阳银行,中信银行,平安银行,宁波银行,江阴银行,张家港行,郑州银行,青岛银行,青农商行
交通运输,40,   白云机场,上海机场,日照港,上港集团,中远海能,南方航空,东方航空,铁龙物流,建发股份,海航控股,圆通速递,山东高速,中远海特,淮河能源,物产中大,天津港,中储股份,唐山港,大秦铁路,宁波港,春秋航空,中国国航,广州港,青岛港,广深铁路,中国外运,京沪高铁,中远海发,大连港,中远海控,招商南油,德邦股份,吉祥航空,深圳机场,招商港口,传化智联,韵达股份,怡亚通,顺丰控股,申通快递
汽车,32,   东风汽车,宇通客车,上汽集团,福田汽车,广汇汽车,国机汽车,江淮汽车,福耀玻璃,均胜电子,北汽蓝谷,华域汽车,小康股份,广汽集团,长城汽车,拓普集团,郑煤机,星宇股份,东方时尚,隆鑫通用,科博达,特力A,潍柴动力,万向钱潮,威孚高科,长安汽车,一汽解放,中鼎股份,宁波华翔,万丰奥威,万里扬,比亚迪,光启技术
公用事业,31,   首创股份,华能国际,上海电力,华能水电,华电国际,联美控股,申能股份,川投能源,国电电力,内蒙华电,创业环保,国投电力,长江电力,贵州燃气,重庆燃气,节能风电,深圳燃气,上海环境,中国核电,伟明环保,深圳能源,皖能电力,兴蓉环境,建投能源,中山公用,宝新能源,启迪环境,湖北能源,盈峰环境,中国广核,碧水源
钢铁,21,   包钢股份,宝钢股份,山东钢铁,杭钢股份,南钢股份,酒钢宏兴,方大特钢,新钢股份,马钢股份,柳钢股份,重庆钢铁,中信特钢,河钢股份,韶钢松山,新兴铸管,太钢不锈,鞍钢股份,华菱钢铁,首钢股份,沙钢股份,三钢闽光
化工,52,   中国石化,兴发集团,金发科技,巨化股份,中国巨石,广汇能源,嘉化能源,亿利洁能,安迪苏,万华化学,上海家化,恒力石化,浙江龙盛,三友化工,华鲁恒升,扬农化工,中化国际,华谊集团,君正集团,桐昆股份,滨化股份,玲珑轮胎,和邦生物,新凤鸣,宏和科技,三美股份,珀莱雅,彤程新材,丸美股份,东方盛虹,安道麦A,恒逸石化,鲁西化工,诚志股份

In [4]:
start_time = pd.Timestamp("2020-05-01 00:00:01", tz="Asia/Shanghai")
market_breath = {}
for k, v in stock_candle_data_map.items():
    for i in range(0, len(v.index())):
        close = v.close()[i]
        ma20 = v.raw_data['MA20'][i]
        current_day = v.index()[i]
        sector = stock_sector_map[k]
#         if current_day < start_time:
#             continue
        current_day_data = market_breath.setdefault(str(current_day), {})

        value = current_day_data.setdefault(sector, {})
        all_value = current_day_data.setdefault('all', {})
        above = value.setdefault('above', 0)
        stocks_count = value.setdefault('all', 0)
        
        all_above = all_value.setdefault('above', 0)
        all_stocks_count = all_value.setdefault('all', 0)
        if close > ma20:
            current_day_data[sector]['above'] = above + 1
            current_day_data['all']['above'] = all_above + 1
        value['all'] = stocks_count + 1
        all_value['all'] = all_stocks_count + 1
        
        
for k, v in market_breath.items():
    for name, value in v.items():
        value['percent'] = round(float(value['above'])*100/float(value['all']),2)
market_breath_data_json = json.dumps(market_breath, ensure_ascii=False)
print(len(market_breath_data_json))

153816


In [5]:
print('时间\t' + '\t'.join(sectors) + '\t全部')
# for k, v in market_breath.items():
for k, v in sorted(list(market_breath.items()), key=lambda x:x[0].lower(), reverse=True):
    result = []
    for sector in sectors:
        result.append(str(v.get(sector, {}).get('percent', 0)))
    result.append(str(v.get('all',{}).get('percent', 0)))
    print(k + '\t' + '\t'.join(result))

时间	传媒	钢铁	农林牧渔	休闲服务	计算机	公用事业	银行	家用电器	通信	化工	电子	医药生物	机械设备	有色金属	建筑材料	汽车	轻工制造	食品饮料	交通运输	采掘	纺织服装	电气设备	非银金融	商业贸易	房地产	综合	建筑装饰	国防军工	全部
2020-08-07 00:00:00+08:00	34.21	28.57	81.25	60.0	27.78	25.81	23.53	58.33	15.79	63.46	43.86	48.75	40.0	85.29	91.67	62.5	66.67	30.77	37.5	30.0	50.0	58.62	30.36	23.81	19.05	50.0	31.58	100.0	44.31
2020-08-06 00:00:00+08:00	44.74	52.38	87.5	60.0	44.44	38.71	26.47	83.33	26.32	75.0	61.4	66.25	56.0	91.18	91.67	75.0	66.67	53.85	52.5	40.0	50.0	72.41	75.0	28.57	23.81	75.0	57.89	100.0	59.07
2020-08-05 00:00:00+08:00	39.47	33.33	87.5	60.0	38.89	51.61	20.59	91.67	26.32	80.77	57.89	71.25	56.0	94.12	91.67	75.0	83.33	73.08	42.5	35.0	50.0	79.31	30.36	38.1	21.43	75.0	47.37	90.91	56.2
2020-08-04 00:00:00+08:00	39.47	23.81	87.5	60.0	27.78	70.97	32.35	83.33	21.05	71.15	57.89	72.5	52.0	88.24	100.0	75.0	66.67	69.23	50.0	50.0	50.0	72.41	41.07	38.1	29.27	75.0	73.68	90.91	57.64
2020-08-03 00:00:00+08:00	50.0	33.33	93.75	40.0	44.44	64.52	0.0	91.67	31.58	75.0	66.67	82.5	68.0	85.29	100.0	65.